# Cell Analysis Pipeline - Interactive Notebook

This notebook runs the complete cell analysis pipeline step by step using PyImageJ and our custom Python modules.

## Pipeline Steps:
1. **Configuration** - Set sample parameters
2. **Initialize PyImageJ** - Load ImageJ environment
3. **Preprocessing** - Run ImageJ macro to generate raw/mask images
4. **Segmentation** - Watershed segmentation to identify cells
5. **Padding** - Extract masked cells and pad to square format
6. **Filtering** - Detect cut-off cells
7. **Manual Review** - GUI for quality control
8. **Load ROIs** - Load ROIs into ImageJ
9. **Extract Channels** - Measure fluorescence in all channels

## Cell 1: Installation & Imports

Run this cell first to install dependencies and import modules.

In [2]:
# Install dependencies (uncomment if needed)
# !pip install pyimagej numpy opencv-python pillow scikit-image scipy pandas

import imagej
import numpy as np
import pandas as pd
from pathlib import Path

# Import our custom pipeline modules
import segment_cells_watershed as segment_cells_watershed
import pad_raw_crops
import filter_bad_cells
import manual_cell_reviewer
import imagej_functions
import extract_roi_crops
import cv2
print("✓ All modules imported successfully!")

# Initialize default PARAMS (will be overwritten by config cell)
# Segmentation method selection
SEGMENTATION_METHOD = 'cellpose'  # Options: 'cellpose' or 'watershed'

PARAMS = {
    'min_distance': 12,
    'min_size': 500,
    'max_size': 9000,
    'cellpose_model': 'cyto2',
    'cellpose_diameter': None,
    'cellpose_flow_threshold': 0.4,
    'cellpose_cellprob_threshold': 0.0,
    'cellpose_use_gpu': True,
    'consecutive_threshold': 20,
    'target_size': 64
}
print("✓ Default PARAMS initialized (run config cell to customize)")


✓ All modules imported successfully!
✓ Default PARAMS initialized (run config cell to customize)


In [ ]:
# Import Cellpose segmentation module (optional - for deep learning segmentation)
try:
    import segment_cells_cellpose
    CELLPOSE_AVAILABLE = True
    print("✓ Cellpose module imported successfully!")
except ImportError as e:
    CELLPOSE_AVAILABLE = False
    print(f"⚠️  Cellpose not available: {e}")
    print("   Install with: pip install cellpose")
    print("   Falling back to watershed segmentation")

In [ ]:
# ============================================================================
# CONFIGURATION 
# ============================================================================

# Segmentation method
SEGMENTATION_METHOD = 'cellpose'  # Options: 'cellpose' or 'watershed'

# Sample to process
BASE_PATH = "/Users/taeeonkong/Desktop/Project/Summer2025/20250729_CLLSaSa/1to10"
SAMPLE_FOLDER = "sample3"  # Options: "sample1", "sample2", "sample3"
IMAGE_NUMBER = "2"         # Options: "1", "2", "3", "4", etc.

# Pipeline parameters
PARAMS = {
    # Segmentation - Watershed
    'min_distance': 12,
    'min_size': 50,
    'max_size': 9000,
    
    # Segmentation - Cellpose (deep learning)
    'cellpose_model': 'cyto2',        # Options: 'cyto2', 'nuclei', 'cpsam'
    'cellpose_diameter': None,        # Auto-detect if None
    'cellpose_flow_threshold': 0.4,
    'cellpose_cellprob_threshold': -6.0,
    'cellpose_use_gpu': True,
    
    # Filtering
    'consecutive_threshold': 20,
    
    # Padding
    'target_size': 64
}

print("Configuration:")
print(f"  Sample: {SAMPLE_FOLDER}/{IMAGE_NUMBER}")
print(f"  Base path: {BASE_PATH}")
print(f"  Parameters: {PARAMS}")

## Cell 3: Initialize PyImageJ

**Note:** This may take 1-2 minutes on first run while downloading Java libraries.

In [ ]:
print("Initializing ImageJ with Fiji (this may take 1-2 minutes on first run)...")
print("Downloading Fiji components if needed...")

# Initialize with Fiji to enable macro support
ij = imagej.init('sc.fiji:fiji')

print(f"✓ ImageJ version: {ij.getVersion()}")
print("✓ PyImageJ ready with macro support!")

## Cell 4: Step 0 - Preprocessing (ImageJ Macro)

**Optional:** Run the preprocessing macro to generate `*_raw.jpg` and `*_mask.jpg` files.

Skip this if you've already run `process_actin_fitc.ijm` manually.

In [ ]:
print("="*60)
print("STEP 0: PREPROCESSING")
print("="*60)

result = imagej_functions.preprocess_actin(
    sample_folder=SAMPLE_FOLDER,
    image_number=IMAGE_NUMBER,
    base_path=BASE_PATH,
    ij=ij,
    verbose=True
)

if result['success']:
    print(f"\n✓ Raw file: {result['raw_file']}")
    print(f"✓ Mask file: {result['mask_file']}")
else:
    print(f"\n✗ Error: {result['error']}")

## Cell 5-alt: Step 1 - Cell Segmentation with Cellpose (Alternative)

**Use Cellpose deep learning model for cell segmentation instead of watershed.**

### Benefits:
- Better boundary detection with deep learning
- Handles complex/irregular cell shapes
- Pre-trained on thousands of cell images
- Auto-detects cell diameter

### Output Folders (separate from watershed):
- `cell_rois_cellpose/` - ROI masks
- `raw_crops_cellpose/` - Raw cell crops
- `original_image_cellpose.tif` - Reference image

**Note:** Requires `pip install cellpose`. First run will download the model (~100-200MB).

In [ ]:
if SEGMENTATION_METHOD != 'cellpose':
    print("⏭️  Skipping Cellpose segmentation (SEGMENTATION_METHOD = '{0}')".format(SEGMENTATION_METHOD))
elif not CELLPOSE_AVAILABLE:
    print("⚠️  Cellpose not available. Please install with: pip install cellpose")
    print("   Or change SEGMENTATION_METHOD to 'watershed'")
else:
    print("\n" + "="*60)
    print("STEP 1: CELLPOSE CELL SEGMENTATION")
    print("="*60)

    result = segment_cells_cellpose.segment_cells_cellpose(
        sample_folder=SAMPLE_FOLDER,
        image_number=IMAGE_NUMBER,
        base_path=BASE_PATH,
        model_type=PARAMS['cellpose_model'],
        diameter=PARAMS['cellpose_diameter'],
        flow_threshold=PARAMS['cellpose_flow_threshold'],
        cellprob_threshold=PARAMS['cellpose_cellprob_threshold'],
        min_size=PARAMS['min_size'],
        max_size=PARAMS['max_size'],
        use_gpu=PARAMS['cellpose_use_gpu'],
        verbose=True
    )

    if result['success']:
        print(f"\n✓ Segmented {result['num_cells']} cells with Cellpose")
        print(f"  ROI dir: {result['roi_dir']}")
        print(f"  Raw crops dir: {result['raw_crops_dir']}")
    else:
        print(f"\n✗ Error: {result['error']}")

if SEGMENTATION_METHOD != 'watershed':
    print("⏭️  Skipping Watershed segmentation (SEGMENTATION_METHOD = '{0}')".format(SEGMENTATION_METHOD))
else:
    print("\n" + "="*60)
    print("STEP 1: CELL SEGMENTATION (WATERSHED)")
    print("="*60)

    result = segment_cells.segment_cells(
        sample_folder=SAMPLE_FOLDER,
        image_number=IMAGE_NUMBER,
        base_path=BASE_PATH,
        min_distance=PARAMS['min_distance'],
        min_size=PARAMS['min_size'],
        max_size=PARAMS['max_size'],
        verbose=True
    )

    if result['success']:
        print(f"\n✓ Segmented {result['num_cells']} cells")
        print(f"  ROI dir: {result['roi_dir']}")
        print(f"  Raw crops dir: {result['raw_crops_dir']}")
    else:
        print(f"\n✗ Error: {result['error']}")

In [ ]:
# Visualize segmentation results with boundaries
if SEGMENTATION_METHOD == 'cellpose':
    import matplotlib.pyplot as plt
    from pathlib import Path

    # Check if segmentation outputs exist
    base_dir = f"{BASE_PATH}/{SAMPLE_FOLDER}/{IMAGE_NUMBER}"
    roi_dir = Path(base_dir) / "cell_rois"

    if roi_dir.exists():
        from PIL import Image
        import numpy as np
        
        # Load original image
        raw_path = list(Path(base_dir).glob("*_raw.jpg"))[0]
        raw_img = np.array(Image.open(raw_path).convert('L'))
        
        # Load all ROI masks and combine
        roi_files = sorted(roi_dir.glob("*.tif"))
        combined_mask = np.zeros_like(raw_img)
        
        for i, roi_file in enumerate(roi_files):
            roi_mask = np.array(Image.open(roi_file).convert('L'))
            combined_mask[roi_mask > 0] = i + 1
        
        # Create visualization with boundaries
        from skimage.segmentation import find_boundaries
        boundaries = find_boundaries(combined_mask, mode='inner')
        
        # Overlay boundaries on raw image
        overlay = np.stack([raw_img] * 3, axis=-1)
        overlay[boundaries] = [255, 0, 0]  # Red boundaries
        
        # Display
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))
        
        axes[0].imshow(raw_img, cmap='gray')
        axes[0].set_title('Original Image')
        axes[0].axis('off')
        
        axes[1].imshow(combined_mask, cmap='nipy_spectral')
        axes[1].set_title(f'Cellpose Segmentation ({len(roi_files)} cells)')
        axes[1].axis('off')
        
        axes[2].imshow(overlay)
        axes[2].set_title('Boundaries Overlay')
        axes[2].axis('off')
        
        plt.tight_layout()
        
        print(f"✓ Visualization complete: {len(roi_files)} cells segmented")
    else:
        print(f"⚠️  No segmentation outputs found in {roi_dir}")
        print("   Please run Cell 5-alt (Cellpose segmentation) first.")
else:
    print(f"⏭️  Skipping visualization (SEGMENTATION_METHOD = '{SEGMENTATION_METHOD}')")

## Visualize Cellpose Segmentation Results

View the segmentation overlay to verify cell detection quality.

## Cell 5b: Extract ROI Crops

Extract actual cell images from original Actin-FITC using ROI masks (non-square, exact bounding box).

In [ ]:
print("\n" + "="*60)
print("EXTRACT ROI CROPS FROM TIF")
print("="*60)

result = extract_roi_crops.extract_roi_crops(
    sample_folder=SAMPLE_FOLDER,
    image_number=IMAGE_NUMBER,
    base_path=BASE_PATH,
    source_image="Actin-FITC.tif",
    output_dir_name="roi_crops_tif",
    verbose=True
)

if result['success']:
    print(f"\n✓ Extracted {result['num_extracted']} cell crops from TIF")
    print(f"  Output dir: {result['output_dir']}")
else:
    print(f"\n✗ Error: {result['error']}")

## Cell 5c: Extract ROI Crops as PNG

Extract cell images as PNG with transparent background (great for viewing in Preview).

In [ ]:
print("\n" + "="*60)
print("EXTRACT ROI CROPS AS PNG (WITH TRANSPARENCY)")
print("="*60)

# Find the raw JPG file
import glob
base_dir_path = f"{BASE_PATH}/{SAMPLE_FOLDER}/{IMAGE_NUMBER}"
raw_jpg_files = glob.glob(f"{base_dir_path}/*_raw.jpg")

if raw_jpg_files:
    raw_jpg = raw_jpg_files[0].split("/")[-1]
    print(f"Using: {raw_jpg}")
    
    result = extract_roi_crops.extract_roi_crops(
        sample_folder=SAMPLE_FOLDER,
        image_number=IMAGE_NUMBER,
        base_path=BASE_PATH,
        source_image=raw_jpg,
        output_dir_name="roi_crops_whiteBg",
        use_transparency=True,
        verbose=True
    )
    
    if result['success']:
        print(f"\n✓ Extracted {result['num_extracted']} cell crops as PNG with transparency")
        print(f"  Output dir: {result['output_dir']}")
    else:
        print(f"\n✗ Error: {result['error']}")
else:
    print("✗ No raw JPG file found. Run preprocessing first!")

## Cell 5d: Extract ROI Crops from JPG

Extract cell images from JPG with white background for easy viewing.

In [ ]:
print("\n" + "="*60)
print("EXTRACT ROI CROPS FROM JPG (WHITE BACKGROUND)")
print("="*60)

# Find the raw JPG file
import glob
import numpy as np
from PIL import Image

base_dir_path = f"{BASE_PATH}/{SAMPLE_FOLDER}/{IMAGE_NUMBER}"
raw_jpg_files = glob.glob(f"{base_dir_path}/*_raw.jpg")

if raw_jpg_files:
    raw_jpg = raw_jpg_files[0].split("/")[-1]
    print(f"Using: {raw_jpg}")
    
    result = extract_roi_crops.extract_roi_crops(
        sample_folder=SAMPLE_FOLDER,
        image_number=IMAGE_NUMBER,
        base_path=BASE_PATH,
        source_image=raw_jpg,
        output_dir_name="roi_crops_jpg",
        use_transparency=False,
        verbose=True
    )
    
    if result['success']:
        print(f"\n✓ Extracted {result['num_extracted']} cell crops from JPG")
        print(f"  Output dir: {result['output_dir']}")
    else:
        print(f"\n✗ Error: {result['error']}")
else:
    print("✗ No raw JPG file found. Run preprocessing first!")

## Cell 9: Step 5 - Load ROIs in ImageJ

Load all cell ROIs onto the original image in ImageJ.

In [ ]:
print("\n" + "="*60)
print("STEP 5: LOAD ROIS IN IMAGEJ")
print("="*60)

result = imagej_functions.load_rois(
    sample_folder=SAMPLE_FOLDER,
    image_number=IMAGE_NUMBER,
    base_path=BASE_PATH,
    ij=ij,
    verbose=True
)

if result['success']:
    print(f"\n✓ Loaded {result['num_rois']} ROIs")
else:
    print(f"\n✗ Error: {result['error']}")

## Cell 10: Step 6 - Extract Channel Measurements

Measure fluorescence intensity in all channels for each cell ROI.

In [ ]:
print("\n" + "="*60)
print("STEP 6: EXTRACT CHANNEL MEASUREMENTS")
print("="*60)

result = imagej_functions.extract_channel_measurements(
    sample_folder=SAMPLE_FOLDER,
    image_number=IMAGE_NUMBER,
    base_path=BASE_PATH,
    ij=ij,
    verbose=True
)

if result['success']:
    print(f"\n✓ Extracted {len(result['measurement_files'])} measurement files")
else:
    print(f"\n✗ Error: {result['error']}")

## Cell 6: Step 2 - Pad Masked Cells

Extract cells using ROI masks and pad to square format.

In [ ]:
print("\n" + "="*60)
print("STEP 2: PAD MASKED CELLS")
print("="*60)

result = pad_raw_crops.pad_masked_cells(
    sample_folder=SAMPLE_FOLDER,
    image_number=IMAGE_NUMBER,
    base_path=BASE_PATH,
    target_size=PARAMS['target_size'],
    verbose=True
)

if result['success']:
    print(f"\n✓ Padded {result['num_padded']} cells to {PARAMS['target_size']}x{PARAMS['target_size']}")
    print(f"  Output dir: {result['output_dir']}")
else:
    print(f"\n✗ Error: {result['error']}")

## Cell 7: Step 3 - Filter Cut-Off Cells

Analyze ROI masks to detect cells that are cut off at image borders.

In [ ]:
print("\n" + "="*60)
print("STEP 3: FILTER CUT-OFF CELLS")
print("="*60)

result = filter_bad_cells.filter_bad_cells(
    sample_folder=SAMPLE_FOLDER,
    image_number=IMAGE_NUMBER,
    base_path=BASE_PATH,
    consecutive_threshold=PARAMS['consecutive_threshold'],
    verbose=True
)

if result['success']:
    print(f"\n✓ Analysis complete:")
    print(f"  WHOLE cells: {result['whole_count']}")
    print(f"  CUT cells: {result['cut_count']}")
    print(f"  CSV saved: {result['output_csv']}")
else:
    print(f"\n✗ Error: {result['error']}")

## Cell 8: Step 4 - Manual Review (Optional)

Launch GUI window for manual cell quality review.

**Controls:**
- ← Left Arrow: Mark as BAD
- → Right Arrow: Mark as GOOD
- B: Undo last classification
- Q/Escape: Quit and save

In [ ]:
# print("\n" + "="*60)
# print("STEP 4: MANUAL REVIEW")
# print("="*60)

# result = manual_cell_reviewer.review_cells(
#     sample_folder=SAMPLE_FOLDER,
#     image_number=IMAGE_NUMBER,
#     base_path=BASE_PATH,
#     verbose=True
# )

# if result['success']:
#     print(f"\n✓ Manual review complete:")
#     print(f"  GOOD cells: {result['good_count']}")
#     print(f"  BAD cells: {result['bad_count']}")
#     print(f"  CSV saved: {result['output_csv']}")
# else:
#     print(f"\n✗ Error: {result['error']}")